In [2]:
%run ../functions_scripts/database_functions.ipynb
%run ../functions_scripts/dataframe_cleaning_functions.ipynb

In [3]:

from pathlib import Path
from typing import List, Optional
from sqlalchemy import create_engine
from sqlalchemy.engine import Engine
from sqlalchemy import text
from psycopg2.extras import execute_values
from sqlalchemy_utils import database_exists, create_database
import pandas as pd

import os
from dotenv import load_dotenv

load_dotenv()

RAW_DIR = Path(os.getenv("RAW_DIR"))
CLEAN_DIR = Path(os.getenv("CLEAN_DIR"))
ADMIN_DB_URL = os.getenv("ADMIN_DB_URL")
DB_URL = os.getenv("DB_URL")



# 🧹 Nettoyage de la table Véhicule 


In [4]:
# Etablir connection avec la base de données
engine = get_engine(DB_URL) 

In [5]:
df_vehicles = get_df_from_table(engine, "bronze", "vehicules_raw")
df_vehicles.head()

,identifiant_de_l_accident,identifiant_vehicule,sens,categorie_vehicule,obstacle_fixe_heurte,obstacle_mobile_heurte,point_de_choc,manoeuvre,nombre_d_occupants
0,201700033619,"B01,A01",None,"VL seul,VL seul",None,"Véhicule,Véhicule","Arrière,Avant",Sans changement de direction,None
1,201700048262,"A01,E01,C01,B01,D01",PK ou PR ou numéro d’adresse postale croissant...,"VL seul,VL seul,VL seul,VL seul,VL seul",None,"Véhicule,Véhicule,Véhicule,Véhicule,Véhicule","Avant droit,Arrière,Côté gauche,Arrière droit,...","Manœuvre d’évitement,Arrêté (hors stationnemen...",None
2,201700048288,"D01,B01,A01,C01",PK ou PR ou numéro d’adresse postale décroissa...,"VL seul,VL seul,VL seul,VL seul",None,"Véhicule,Véhicule,Véhicule,Véhicule","Arrière,Avant,Avant,Arrière","Même sens,Même sens,Même sens,Même sens",None
3,201700048486,"A01,B01",PK ou PR ou numéro d’adresse postale croissant...,"VL seul,VL seul",None,"Véhicule,Véhicule","Avant gauche,Côté gauche","Changeant de file A gauche,Déporté A gauche",None
4,201700048639,"Z01,A01",PK ou PR ou numéro d’adresse postale décroissa...,"VL seul,Motocyclette > 125 cm3",None,"Véhicule,Véhicule",Avant droit,Sans changement de direction,None


# 🧹 Nettoyage de la table vehicule 

In [6]:
# Supprimer les doublons
print(f"Nombre de lignes avant suppression des doublons : {df_vehicles.shape[0]}")
df_vehicles = df_vehicles.drop_duplicates()
print(f"Nombre de lignes après suppression des doublons : {df_vehicles.shape[0]}")
#remplacer les valeurs nulles
remplacer_valeurs_nulles(df_vehicles)
#Nettoyage sémantique
#Traiter valeurs spéciales (-1, 00, NULL chaîne).
df_vehicles =  traiter_valeurs_speciales(df_vehicles)
df_vehicles.head()

Nombre de lignes avant suppression des doublons : 1427733
Nombre de lignes après suppression des doublons : 475911


,identifiant_de_l_accident,identifiant_vehicule,sens,categorie_vehicule,obstacle_fixe_heurte,obstacle_mobile_heurte,point_de_choc,manoeuvre,nombre_d_occupants
0,201700033619,"B01,A01",None,"VL seul,VL seul",None,"Véhicule,Véhicule","Arrière,Avant",Sans changement de direction,None
1,201700048262,"A01,E01,C01,B01,D01",PK ou PR ou numéro d’adresse postale croissant...,"VL seul,VL seul,VL seul,VL seul,VL seul",None,"Véhicule,Véhicule,Véhicule,Véhicule,Véhicule","Avant droit,Arrière,Côté gauche,Arrière droit,...","Manœuvre d’évitement,Arrêté (hors stationnemen...",None
2,201700048288,"D01,B01,A01,C01",PK ou PR ou numéro d’adresse postale décroissa...,"VL seul,VL seul,VL seul,VL seul",None,"Véhicule,Véhicule,Véhicule,Véhicule","Arrière,Avant,Avant,Arrière","Même sens,Même sens,Même sens,Même sens",None
3,201700048486,"A01,B01",PK ou PR ou numéro d’adresse postale croissant...,"VL seul,VL seul",None,"Véhicule,Véhicule","Avant gauche,Côté gauche","Changeant de file A gauche,Déporté A gauche",None
4,201700048639,"Z01,A01",PK ou PR ou numéro d’adresse postale décroissa...,"VL seul,Motocyclette > 125 cm3",None,"Véhicule,Véhicule",Avant droit,Sans changement de direction,None


In [7]:
#Fermeture des connexions actives à la base de données
admin_engine = get_engine(ADMIN_DB_URL)
show_open_connections(admin_engine, DB_NAME)
close_all_connections(admin_engine, DB_NAME)
show_open_connections(admin_engine, DB_NAME)


INFO:__main__:Toutes les connexions à 'db_accident' ont été fermées (sauf la session actuelle).
INFO:__main__:Engine SQLAlchemy libéré — plus aucune connexion active.


(3693, 'postgres', 'db_accident', 'idle', 'ROLLBACK')
